In [7]:
import psycopg2

conn = psycopg2.connect(
    host="localhost",
    port="5433",
    database="krankenhaus",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


In [8]:
drop_tables = """
-- =========================
-- DROP TABLES
-- =========================

DROP TABLE IF EXISTS Event_Medikament CASCADE;
DROP TABLE IF EXISTS Arzt_Event CASCADE;
DROP TABLE IF EXISTS Event CASCADE;
DROP TABLE IF EXISTS Behandlung CASCADE;
DROP TABLE IF EXISTS Medikament CASCADE;
DROP TABLE IF EXISTS Bett CASCADE;
DROP TABLE IF EXISTS Reserve CASCADE;
DROP TABLE IF EXISTS Pflegekraft CASCADE;
DROP TABLE IF EXISTS Arzt CASCADE;
DROP TABLE IF EXISTS Patient CASCADE;
DROP TABLE IF EXISTS Raum CASCADE;
DROP TABLE IF EXISTS Station CASCADE;
DROP TABLE IF EXISTS Krankenhaus CASCADE;
"""



In [9]:
conn.rollback()

cur.execute(drop_tables)
conn.commit()



In [10]:
create_tables = """
CREATE TABLE Krankenhaus (
    KH_ID INT PRIMARY KEY,
    Strasse VARCHAR(100) NOT NULL,
    Hausnummer VARCHAR(10) NOT NULL,
    Postleitzahl VARCHAR(10) NOT NULL,
    Stadt VARCHAR(50) NOT NULL,
    Name VARCHAR(100) NOT NULL,
    CHECK (Postleitzahl ~ '^[0-9]{4}$')
);

CREATE TABLE Station (
    Station_ID INT PRIMARY KEY,
    Name VARCHAR(50) NOT NULL,
    Kapazitaet INT NOT NULL,
    KH_ID INT NOT NULL,
    CHECK (Kapazitaet > 0),
    FOREIGN KEY (KH_ID) REFERENCES Krankenhaus(KH_ID)
);

CREATE TABLE Raum (
    Raumnummer INT PRIMARY KEY,
    Station_ID INT NOT NULL,
    Art VARCHAR(50),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Pflegekraft (
    Pflege_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Station_ID INT NOT NULL,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Patient (
    Patient_ID INT PRIMARY KEY,
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    Versicherungsnummer VARCHAR(30) UNIQUE NOT NULL,
    Geburtsdatum DATE NOT NULL,
    Strasse VARCHAR(100),
    Hausnummer VARCHAR(10),
    Postleitzahl VARCHAR(10),
    Stadt VARCHAR(50),
    Elektronisches_Patientendossier_ID VARCHAR(50) UNIQUE,
    Station_ID INT,
    CHECK (Postleitzahl IS NULL OR Postleitzahl ~ '^[0-9]{4}$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Bett (
    Bett_ID INT PRIMARY KEY,
    Status VARCHAR(20) NOT NULL,
    Patient_ID INT,
    Raum_ID INT NOT NULL,
    CHECK (Status IN ('frei', 'belegt')),
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    FOREIGN KEY (Raum_ID) REFERENCES Raum(Raumnummer)
);

CREATE TABLE Arzt (
    Arzt_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Vorname VARCHAR(50) NOT NULL,
    Nachname VARCHAR(50) NOT NULL,
    ist_extern BOOLEAN NOT NULL,
    Station_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID)
);

CREATE TABLE Reserve (
    Reserve_ID INT PRIMARY KEY,
    Telefonnummer VARCHAR(30),
    Arzt_ID INT,
    Station_ID INT,
    Pflegekraft_ID INT,
    CHECK (Telefonnummer IS NULL OR Telefonnummer ~ '^[0-9+ ]+$'),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Station_ID) REFERENCES Station(Station_ID),
    FOREIGN KEY (Pflegekraft_ID) REFERENCES Pflegekraft(Pflege_ID)
);

CREATE TABLE Behandlung (
    Behandlungs_ID INT PRIMARY KEY,
    abgeschlossen BOOLEAN NOT NULL,
    Beschreibung VARCHAR(255),
    Start_Datum DATE NOT NULL,
    End_Datum DATE,
    Patient_ID INT NOT NULL,
    FOREIGN KEY (Patient_ID) REFERENCES Patient(Patient_ID),
    CHECK (End_Datum IS NULL OR End_Datum >= Start_Datum)
);

CREATE TABLE Medikament (
    Medikament_ID INT PRIMARY KEY,
    Wirkstoff VARCHAR(100),
    Name VARCHAR(100) NOT NULL   
);

CREATE TABLE Event (
    Event_ID INT PRIMARY KEY,
    Beschreibung VARCHAR(255),
    Uhrzeit TIME NOT NULL,
    Datum DATE NOT NULL,
    RaumID INT NOT NULL,
    Behandlung_ID INT NOT NULL,
    FOREIGN KEY (RaumID) REFERENCES Raum(Raumnummer),
    FOREIGN KEY (Behandlung_ID) REFERENCES Behandlung(Behandlungs_ID)
);


CREATE TABLE Arzt_Event (
    Arzt_ID INT NOT NULL,
    Event_ID INT NOT NULL,
    PRIMARY KEY (Arzt_ID, Event_ID),
    FOREIGN KEY (Arzt_ID) REFERENCES Arzt(Arzt_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID)
);

CREATE TABLE Event_Medikament (
    Event_ID INT NOT NULL,
    Medikament_ID INT NOT NULL,
    PRIMARY KEY (Event_ID, Medikament_ID),
    FOREIGN KEY (Event_ID) REFERENCES Event(Event_ID),
    FOREIGN KEY (Medikament_ID) REFERENCES Medikament(Medikament_ID)
);
"""
cur.execute(create_tables)
conn.commit()


In [11]:
import random
import datetime
from faker import Faker

fake = Faker("de_CH")
random.seed(1)

TODAY = datetime.date(2025, 12, 12)

def phone(i):
    return f"+41 79 {i:06d}"

# =========================
# HARDCODED DATA
# =========================

BEHANDLUNG_CASES = [
    "Appendektomie Nachkontrolle",
    "Blutdruckeinstellung",
    "Diabetes Verlaufskontrolle",
    "Atemwegsinfekt Behandlung",
    "Herz-Kreislauf Check",
    "Wundversorgung",
    "Schmerztherapie",
    "Magen-Darm Untersuchung",
    "Präoperative Abklärung",
    "Postoperative Nachsorge"
]

EVENT_BESCHREIBUNGEN = [
    "Visite",
    "Operation",
    "Nachkontrolle",
    "Medikamentengabe",
    "Notfallbehandlung",
    "Diagnostische Untersuchung",
    "Pflegevisite",
    "Therapieanpassung",
    "Laborbesprechung",
    "Entlassungsgespräch"
]

STATION_NAMES = [
    "Chirurgie",
    "Innere Medizin",
    "Pädiatrie",
    "Notfall",
    "Intensivstation"
]

RAUM_ARTEN = [
    "Operationssaal",
    "Untersuchungsraum",
    "Patientenzimmer",
    "Aufenthaltsraum"
]

MEDIKAMENTE = [
    (1, "Paracetamol", "Paracetamol"),
    (2, "Ibuprofen", "Ibuprofen"),
    (3, "Aspirin", "Acetylsalicylsäure"),
    (4, "Amoxicillin", "Amoxicillin"),
    (5, "Insulin", "Insulin")
]

# =========================
# Krankenhaus (3)
# =========================
for kh_id in range(1, 4):
    cur.execute("""
        INSERT INTO Krankenhaus
        VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        kh_id,
        fake.street_name(),
        fake.building_number(),
        f"{1000 + kh_id}",
        fake.city(),
        f"Kantonsspital {kh_id}"
    ))
conn.commit()

# =========================
# Station (15)
# =========================
station_ids = []
sid = 1
for kh_id in range(1, 4):
    for name in STATION_NAMES:
        cur.execute("""
            INSERT INTO Station
            VALUES (%s,%s,%s,%s)
        """, (sid, name, random.randint(30, 65), kh_id))
        station_ids.append(sid)
        sid += 1
conn.commit()

# =========================
# Raum
# =========================
raum_rows = []
raum_id = 1

for sid in station_ids:
    for art in RAUM_ARTEN:
        count = 4 if art == "Patientenzimmer" else 2
        for _ in range(count):
            cur.execute("""
                INSERT INTO Raum
                VALUES (%s,%s,%s)
            """, (raum_id, sid, art))
            raum_rows.append((raum_id, art))
            raum_id += 1
conn.commit()

# =========================
# Medikament
# =========================
for mid, name, wirkstoff in MEDIKAMENTE:
    cur.execute("""
        INSERT INTO Medikament
        VALUES (%s,%s,%s)
    """, (mid, wirkstoff, name))
conn.commit()

# =========================
# Patient
# =========================
NUM_PATIENTS = 150
for pid in range(1, NUM_PATIENTS + 1):
    cur.execute("""
        INSERT INTO Patient
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    """, (
        pid,
        fake.first_name(),
        fake.last_name(),
        f"CH{pid:08d}",
        fake.date_of_birth(minimum_age=0, maximum_age=95),
        fake.street_name(),
        fake.building_number(),
        f"{1000 + pid % 50:04d}",
        fake.city(),
        f"EPD-{pid}",
        random.choice(station_ids)
    ))
conn.commit()

# =========================
# Arzt
# =========================
NUM_AERZTE = 24
for aid in range(1, NUM_AERZTE + 1):
    cur.execute("""
        INSERT INTO Arzt
        VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        aid,
        phone(aid),
        fake.first_name(),
        fake.last_name(),
        aid % 7 == 0,
        random.choice(station_ids)
    ))
conn.commit()

# =========================
# 🛏 BETT (NUR SINNVOLLE RÄUME)
# =========================
bett_id = 1

cur.execute("""
    SELECT Raumnummer
    FROM Raum
    WHERE Art IN ('Patientenzimmer', 'Aufenthaltsraum')
""")
bett_raeume = [r[0] for r in cur.fetchall()]

patient_iter = iter(range(1, NUM_PATIENTS + 1))

for raum_id in bett_raeume:
    for _ in range(random.randint(2, 4)):
        try:
            pid = next(patient_iter)
            status = "belegt"
        except StopIteration:
            pid = None
            status = "frei"

        cur.execute("""
            INSERT INTO Bett
            VALUES (%s,%s,%s,%s)
        """, (bett_id, status, pid, raum_id))
        bett_id += 1

conn.commit()

# =========================
# Behandlung
# =========================
NUM_BEHANDLUNGEN = 220
for bid in range(1, NUM_BEHANDLUNGEN + 1):
    abgeschlossen = random.random() > 0.35
    if abgeschlossen:
        start = fake.date_between(datetime.date(2022, 1, 1), TODAY)
        end = min(start + datetime.timedelta(days=random.randint(1, 30)), TODAY)
    else:
        start = TODAY + datetime.timedelta(days=random.randint(1, 90))
        end = None

    cur.execute("""
        INSERT INTO Behandlung
        VALUES (%s,%s,%s,%s,%s,%s)
    """, (
        bid,
        abgeschlossen,
        random.choice(BEHANDLUNG_CASES),
        start,
        end,
        random.randint(1, NUM_PATIENTS)
    ))
conn.commit()

# =========================
# Events
# =========================
cur.execute("SELECT Behandlungs_ID FROM Behandlung;")
behandlungen = [b[0] for b in cur.fetchall()]
valid_rooms = [r[0] for r in raum_rows if r[1] != "Operationssaal"]

event_id = 1
for bid in behandlungen:
    for _ in range(random.randint(2, 4)):
        cur.execute("""
            INSERT INTO Event
            VALUES (%s,%s,%s,%s,%s,%s)
        """, (
            event_id,
            random.choice(EVENT_BESCHREIBUNGEN),
            fake.time(),
            TODAY + datetime.timedelta(days=random.randint(1, 90)),
            random.choice(valid_rooms),
            bid
        ))
        event_id += 1
conn.commit()

# =========================
# GARANTIE: JEDER ARZT HAT HEUTE EVENTS
# =========================
for arzt_id in range(1, NUM_AERZTE + 1):
    for _ in range(5):
        bid = random.choice(behandlungen)

        cur.execute("""
            INSERT INTO Event
            VALUES (%s,%s,%s,%s,%s,%s)
        """, (
            event_id,
            "Visite",
            fake.time(),
            TODAY,
            random.choice(valid_rooms),
            bid
        ))

        cur.execute("""
            INSERT INTO Arzt_Event
            VALUES (%s,%s)
        """, (arzt_id, event_id))

        event_id += 1

conn.commit()

# =========================
# Event_Medikament
# =========================
for eid in range(1, event_id):
    for mid in random.sample([1,2,3,4,5], random.randint(1,2)):
        cur.execute("""
            INSERT INTO Event_Medikament
            VALUES (%s,%s)
        """, (eid, mid))
conn.commit()

print("✔ Datenbank vollständig, realistisch & konsistent befüllt")


✔ Datenbank vollständig, realistisch & konsistent befüllt


In [12]:
conn.rollback()
